# Thesis Notebook — OULAD (Open University Learning Analytics Dataset)

This notebook shows the full thesis pipeline using the **OULAD** dataset from Kaggle.

What it contains:
- Data import & merging of OULAD tables
- Robust error handling for missing files
- Exploratory Data Analysis (EDA) and visualizations
- Preprocessing & feature engineering (per-student aggregation)
- Dimensionality reduction (PCA, optional MCA/FAMD)
- Feature selection (RFE, Genetic Algorithm, PSO-style)
- Unsupervised clustering (Popularity Architecture)
- Supervised models (RandomForest, KNN, XGBoost if available)
- Neural network architectures (multiple configs)
- Explainable AI (SHAP, and LIME if present)
- Output saving (figures and CSVs)

**Important:** This notebook will attempt to download the dataset using the Kaggle API if a `kaggle.json` file is placed at `~/.kaggle/kaggle.json`. If you prefer manual download, unzip the dataset into `/mnt/data/OULAD` before running the notebook.


In [22]:
# Imports & global configuration
import os, sys, warnings
warnings.filterwarnings('ignore')
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
sns.set(style='whitegrid')

from zipfile import ZipFile
from pathlib import Path
RANDOM_STATE = 42
OUTPUT_DIR = '/content/outputs'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print('Outputs will be saved to', OUTPUT_DIR)


Outputs will be saved to /content/outputs


In [9]:
# Kaggle download: tries to download thedevastator/open-university-learning-analytics-dataset
# Requires kaggle package and a valid ~/.kaggle/kaggle.json token.
DATA_DIR = '/content/OULAD'
os.makedirs(DATA_DIR, exist_ok=True)

def download_kaggle_dataset():
    try:
        import kaggle
    except Exception as e:
        print('kaggle package not installed. Install it with: pip install kaggle')
        return False
    kaggle_json = os.path.expanduser('/content/kaggle.json')
    if not os.path.exists(kaggle_json):
        print('No kaggle.json token found at ~/.kaggle/kaggle.json. Please place it there to enable API download.')
        return False
    try:
        print('Downloading dataset via Kaggle API...')
        os.system('kaggle datasets download -d thedevastator/open-university-learning-analytics-dataset -p /content/mnt/data --unzip')
        # After download, files should be under /mnt/data (unzipped).
        # Move expected CSVs into DATA_DIR if needed
        possible_files = [p for p in Path('/content/mnt/data').glob('**/*') if p.is_file()]
        for p in possible_files:
            if p.suffix.lower() in ['.csv']:
                # copy into DATA_DIR if not already
                dest = Path(DATA_DIR) / p.name
                if not dest.exists():
                    try:
                        dest.write_bytes(p.read_bytes())
                    except Exception:
                        pass
        print('Kaggle download attempt complete. Check', DATA_DIR)
        return True
    except Exception as e:
        print('Kaggle download failed:', e)
        return False

print('If you have not yet placed kaggle.json, do that now. The cell will try to download if possible.')
# Uncomment the next line to attempt automatic download when you're ready.
download_kaggle_dataset()


If you have not yet placed kaggle.json, do that now. The cell will try to download if possible.
kaggle package not installed. Install it with: pip install kaggle


False

In [10]:
# Install kaggle if not yet installed
!pip install kaggle

# Authenticate (make sure kaggle.json is uploaded to ~/.kaggle/)
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the OULAD dataset from Kaggle
!kaggle datasets download -d thedevastator/open-university-learning-analytics-dataset -p /content

# Unzip it
!unzip /content/open-university-learning-analytics-dataset.zip -d /content/OULAD


Dataset URL: https://www.kaggle.com/datasets/thedevastator/open-university-learning-analytics-dataset
License(s): other
  0% 0.00/42.2M [00:00<?, ?B/s]
100% 42.2M/42.2M [00:00<00:00, 1.13GB/s]
Archive:  /content/open-university-learning-analytics-dataset.zip
  inflating: /content/OULAD/uci-open-university-learning-analytics-dataset/OULAD.names  
  inflating: /content/OULAD/uci-open-university-learning-analytics-dataset/assessments.csv  
  inflating: /content/OULAD/uci-open-university-learning-analytics-dataset/courses.csv  
  inflating: /content/OULAD/uci-open-university-learning-analytics-dataset/studentAssessment.csv  
  inflating: /content/OULAD/uci-open-university-learning-analytics-dataset/studentInfo.csv  
  inflating: /content/OULAD/uci-open-university-learning-analytics-dataset/studentRegistration.csv  
  inflating: /content/OULAD/uci-open-university-learning-analytics-dataset/studentVle.csv  
  inflating: /content/OULAD/uci-open-university-learning-analytics-dataset/vle.csv  


In [12]:
# Load OULAD CSVs from DATA_DIR. If you manually placed files, ensure they exist in /mnt/data/OULAD
DATA_DIR = '/content/OULAD/uci-open-university-learning-analytics-dataset'
expected_files = ['studentInfo.csv','studentAssessment.csv','studentVle.csv','courses.csv','assessments.csv','vle.csv']

found = {}
for fname in expected_files:
    fpath = os.path.join(DATA_DIR, fname)
    if os.path.exists(fpath):
        try:
            found[fname] = pd.read_csv(fpath)
            print(f'Loaded {fname} shape', found[fname].shape)
        except Exception as e:
            print('Failed to read', fname, e)
    else:
        print('Missing', fname, 'in', DATA_DIR)

# # Fallback: try to locate any CSVs under /mnt/data that look like OULAD
# if len(found)==0:
#     print('No expected files found in', DATA_DIR, '. Trying to locate OULAD CSVs under /mnt/data...')
#     for p in Path('/mnt/data').glob('**/*.csv'):
#         name = p.name.lower()
#         if 'student' in name or 'vle' in name or 'assessment' in name or 'course' in name:
#             try:
#                 df = pd.read_csv(p)
#                 found[p.name] = df
#                 print('Auto-loaded', p.name, 'shape', df.shape)
#             except Exception as e:
#                 print('Failed to auto-load', p, e)

# Quick check
for k,v in found.items():
    print(k, v.shape)


Loaded studentInfo.csv shape (32593, 12)
Loaded studentAssessment.csv shape (173912, 5)
Loaded studentVle.csv shape (10655280, 6)
Loaded courses.csv shape (22, 3)
Loaded assessments.csv shape (206, 6)
Loaded vle.csv shape (6364, 6)
studentInfo.csv (32593, 12)
studentAssessment.csv (173912, 5)
studentVle.csv (10655280, 6)
courses.csv (22, 3)
assessments.csv (206, 6)
vle.csv (6364, 6)


In [16]:
import pandas as pd
import numpy as np

# --- Helper function ---
def safe_get(found, name):
    """
    Safely fetch a DataFrame from 'found' dict, trying both 'file.csv' and 'file'.
    Returns None if not found.
    """
    out = found.get(name)
    if out is None:
        out = found.get(name.replace('.csv',''))
    return out

# --- Load datasets ---
studentInfo      = safe_get(found, 'studentInfo.csv')
studentAssessment = safe_get(found, 'studentAssessment.csv')
studentVle       = safe_get(found, 'studentVle.csv')
assessments      = safe_get(found, 'assessments.csv')

# --- Aggregate VLE (Virtual Learning Environment interactions) ---
if studentVle is not None:
    # make sure clicks are numeric
    if 'sum_click' in studentVle.columns:
        studentVle['sum_click'] = pd.to_numeric(studentVle['sum_click'], errors='coerce').fillna(0)
        vle_agg = studentVle.groupby('id_student').agg(
            clicks_total=('sum_click','sum'),
            days_active=('date','nunique')
        ).reset_index()
    else:
        # try to auto-detect click column
        possible_clicks = [c for c in studentVle.columns if 'click' in c.lower()]
        if possible_clicks:
            ccol = possible_clicks[0]
            studentVle[ccol] = pd.to_numeric(studentVle[ccol], errors='coerce').fillna(0)
            vle_agg = studentVle.groupby('id_student').agg(
                clicks_total=(ccol,'sum'),
                days_active=('date','nunique') if 'date' in studentVle.columns else ('id_site','nunique')
            ).reset_index()
        else:
            vle_agg = pd.DataFrame({'id_student':[], 'clicks_total':[], 'days_active':[]})
else:
    vle_agg = pd.DataFrame({'id_student':[], 'clicks_total':[], 'days_active':[]})

# --- Aggregate Assessments ---
if studentAssessment is not None:
    if 'score' in studentAssessment.columns:
        # force numeric conversion
        studentAssessment['score'] = pd.to_numeric(studentAssessment['score'], errors='coerce')
        studentAssessment['id_assessment'] = pd.to_numeric(studentAssessment['id_assessment'], errors='coerce')

        assess_agg = studentAssessment.groupby('id_student').agg(
            avg_score=('score','mean'),
            n_assessments=('id_assessment','nunique')
        ).reset_index()
    else:
        assess_agg = pd.DataFrame({'id_student':[], 'avg_score':[], 'n_assessments':[]})
else:
    assess_agg = pd.DataFrame({'id_student':[], 'avg_score':[], 'n_assessments':[]})

# --- Merge all into studentInfo ---
if studentInfo is None:
    raise ValueError("studentInfo.csv not found. Cannot proceed with merge.")

df = studentInfo.copy()

# Merge VLE aggregates
df = df.merge(vle_agg, on='id_student', how='left')

# Merge assessment aggregates
df = df.merge(assess_agg, on='id_student', how='left')

# --- Fill NaNs in new columns ---
for col in ['clicks_total','days_active','avg_score','n_assessments']:
    if col in df.columns:
        df[col] = df[col].fillna(0)

# --- Encode final_result ---
if 'final_result' in df.columns:
    df['target'] = df['final_result'].map({
        'Withdrawn':0,
        'Fail':1,
        'Pass':2,
        'Distinction':3
    })
else:
    df['target'] = np.nan

print("✅ Final merged dataset shape:", df.shape)
display(df.head())


✅ Final merged dataset shape: (32593, 17)


,code_module,code_presentation,id_student,gender,region,highest_education,imd_band,age_band,num_of_prev_attempts,studied_credits,disability,final_result,clicks_total,days_active,avg_score,n_assessments,target
0,AAA,2013J,11391,M,East Anglian Region,HE Qualification,90-100%,55<=,0,240,N,Pass,934.0,40.0,82.0,5.0,2
1,AAA,2013J,28400,F,Scotland,HE Qualification,20-30%,35-55,0,60,N,Pass,1435.0,80.0,66.4,5.0,2
2,AAA,2013J,30268,F,North Western Region,A Level or Equivalent,30-40%,35-55,0,60,Y,Withdrawn,281.0,12.0,0.0,0.0,0
3,AAA,2013J,31604,F,South East Region,A Level or Equivalent,50-60%,35-55,0,60,N,Pass,2158.0,123.0,76.0,5.0,2
4,AAA,2013J,32885,F,West Midlands Region,Lower Than A Level,50-60%,0-35,0,60,N,Pass,1034.0,70.0,54.4,5.0,2


In [17]:
# EDA: missing values, distributions and correlations
def save_fig(fig, name):
    p = os.path.join(OUTPUT_DIR, name)
    fig.savefig(p, bbox_inches='tight')
    print('Saved', p)

print('\n-- Missing values per column --')
print(df.isnull().sum().sort_values(ascending=False).head(30))

# Target distribution
if 'target' in df.columns:
    fig = plt.figure(figsize=(6,4))
    sns.countplot(x='target', data=df)
    plt.title('Target distribution (final_result coded)')
    save_fig(fig, 'target_distribution.png')
    plt.close(fig)

# Numeric distributions
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
num_cols = [c for c in num_cols if c not in ['id_student','target','id_course']]
for c in num_cols[:10]:
    fig = plt.figure(figsize=(6,3))
    sns.histplot(df[c].dropna(), bins=40, kde=True)
    plt.title(f'Distribution: {c}')
    save_fig(fig, f'hist_{c}.png')
    plt.close(fig)

# Correlation heatmap for numeric columns
if len(num_cols) > 1:
    fig = plt.figure(figsize=(10,8))
    sns.heatmap(df[num_cols].corr(), cmap='coolwarm', annot=False)
    plt.title('Numeric correlation matrix')
    save_fig(fig, 'correlation_heatmap.png')
    plt.close(fig)



-- Missing values per column --
code_module             0
code_presentation       0
id_student              0
gender                  0
region                  0
highest_education       0
imd_band                0
age_band                0
num_of_prev_attempts    0
studied_credits         0
disability              0
final_result            0
clicks_total            0
days_active             0
avg_score               0
n_assessments           0
target                  0
dtype: int64
Saved ~/mnt/data/oulad_outputs/target_distribution.png
Saved ~/mnt/data/oulad_outputs/hist_num_of_prev_attempts.png
Saved ~/mnt/data/oulad_outputs/hist_studied_credits.png
Saved ~/mnt/data/oulad_outputs/hist_clicks_total.png
Saved ~/mnt/data/oulad_outputs/hist_days_active.png
Saved ~/mnt/data/oulad_outputs/hist_avg_score.png
Saved ~/mnt/data/oulad_outputs/hist_n_assessments.png
Saved ~/mnt/data/oulad_outputs/correlation_heatmap.png


In [18]:
# Preprocessing: encode categorical, impute and scale numeric features
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

# Drop columns not needed
drop_cols = ['id_student','final_result'] if 'final_result' in df.columns else ['id_student']
drop_cols = [c for c in drop_cols if c in df.columns]
X = df.drop(columns=drop_cols + ['target'] if 'target' in df.columns else drop_cols, errors='ignore')
y = df['target'] if 'target' in df.columns else None

# Identify categorical and numeric
cat_cols = X.select_dtypes(include=['object','category']).columns.tolist()
num_cols = X.select_dtypes(include=[np.number]).columns.tolist()

# Simple encoding for categoricals: label encode small cardinality, one-hot for larger
X_enc = X.copy()
le_dict = {}
for c in cat_cols:
    try:
        if X_enc[c].nunique() <= 10:
            le = LabelEncoder()
            X_enc[c] = X_enc[c].astype(str).fillna('NA')
            X_enc[c] = le.fit_transform(X_enc[c])
            le_dict[c] = le
        else:
            X_enc = pd.get_dummies(X_enc, columns=[c], drop_first=True)
    except Exception as e:
        print('Encoding failed for', c, e)

# Impute numeric missing values and scale
imp = SimpleImputer(strategy='median')
scaler = StandardScaler()

if len(num_cols) > 0:
    X_enc[num_cols] = imp.fit_transform(X_enc[num_cols])
    X_enc[num_cols] = scaler.fit_transform(X_enc[num_cols])

print('Final feature matrix shape:', X_enc.shape)
if y is not None:
    print('Target distribution:', y.value_counts(dropna=False))


Final feature matrix shape: (32593, 34)
Target distribution: target
2    12361
0    10156
1     7052
3     3024
Name: count, dtype: int64


In [19]:
# PCA on numeric features
from sklearn.decomposition import PCA
num_for_pca = [c for c in X_enc.columns if c in num_cols]
if len(num_for_pca) >= 2:
    pca = PCA(n_components=3, random_state=RANDOM_STATE)
    comps = pca.fit_transform(X_enc[num_for_pca])
    print('Explained variance ratios (PCA):', pca.explained_variance_ratio_)
    fig = plt.figure(figsize=(7,6))
    sc = plt.scatter(comps[:,0], comps[:,1], c=(y.fillna(-1) if y is not None else np.zeros(len(comps))), cmap='viridis', alpha=0.6)
    plt.colorbar(sc)
    plt.title('PCA (first 2 components) colored by target')
    save_fig(fig, 'pca_scatter.png')
    plt.close(fig)
else:
    print('Not enough numeric features for PCA.')

# Optional: FAMD/MCA if prince is installed
try:
    import prince
    cat_for_mca = [c for c in X_enc.columns if c not in num_for_pca]
    if len(cat_for_mca) > 0:
        famd = prince.FAMD(n_components=2, random_state=RANDOM_STATE)
        famd_res = famd.fit_transform(X_enc[cat_for_mca].astype(str))
        fig = plt.figure(figsize=(7,6))
        plt.scatter(famd_res[0], famd_res[1], alpha=0.6)
        plt.title('FAMD on categorical features (approx)')
        save_fig(fig, 'famd_scatter.png')
        plt.close(fig)
except Exception as e:
    print('prince not available or FAMD failed:', e)


Explained variance ratios (PCA): [0.47781041 0.19634857 0.1398368 ]
Saved ~/mnt/data/oulad_outputs/pca_scatter.png
prince not available or FAMD failed: No module named 'prince'


In [20]:
# Feature selection: RFE, simple GA, PSO-style
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import random

def rfe_select(X, y, k=12):
    model = LogisticRegression(max_iter=2000, random_state=RANDOM_STATE)
    rfe = RFE(model, n_features_to_select=min(k, X.shape[1]), step=1)
    rfe.fit(X, y)
    return X.columns[rfe.support_].tolist()

def ga_feature_search(X, y, population_size=30, generations=8, k_best=12):
    n = X.shape[1]
    cols = X.columns.tolist()
    def make_individual():
        # ensure at least 1 feature
        mask = [1 if random.random() < 0.5 else 0 for _ in range(n)]
        if sum(mask)==0:
            mask[random.randrange(n)] = 1
        return mask
    def mask_to_feats(mask):
        return [c for c,m in zip(cols, mask) if m==1]
    def fitness(mask):
        feats = mask_to_feats(mask)
        if len(feats)==0:
            return 0.0
        clf = RandomForestClassifier(n_estimators=50, random_state=RANDOM_STATE)
        try:
            scores = cross_val_score(clf, X[feats], y, cv=3, scoring='accuracy', n_jobs=1)
            return float(scores.mean())
        except Exception:
            return 0.0
    pop = [make_individual() for _ in range(population_size)]
    pop_scores = [fitness(p) for p in pop]
    for gen in range(generations):
        # keep top 40%
        zipped = sorted(zip(pop, pop_scores), key=lambda x:-x[1])
        keep = [p for p,s in zipped[:max(2,int(0.4*population_size))]]
        new_pop = keep.copy()
        while len(new_pop) < population_size:
            p1 = random.choice(keep)
            p2 = random.choice(keep)
            point = random.randint(1, n-1)
            child = p1[:point] + p2[point:]
            # mutation
            for i in range(n):
                if random.random() < 0.02:
                    child[i] = 1-child[i]
            new_pop.append(child)
        pop = new_pop
        pop_scores = [fitness(p) for p in pop]
        print(f'GA gen {gen+1}/{generations} best {max(pop_scores):.4f}')
    best = pop[pop_scores.index(max(pop_scores))]
    best_feats = mask_to_feats(best)
    # truncate to k_best by training RF importance if needed
    if len(best_feats) > k_best:
        rf = RandomForestClassifier(n_estimators=50, random_state=RANDOM_STATE)
        rf.fit(X[best_feats], y)
        imp = pd.Series(rf.feature_importances_, index=best_feats).sort_values(ascending=False)
        best_feats = imp.index[:k_best].tolist()
    return best_feats

def pso_feature_search(X, y, n_particles=20, iters=8, k_feats=12):
    n = X.shape[1]
    cols = X.columns.tolist()
    rng = np.random.RandomState(RANDOM_STATE)
    particles = rng.rand(n_particles, n)
    velocities = rng.normal(0, 0.1, size=(n_particles, n))
    pbest = particles.copy()
    pbest_scores = np.zeros(n_particles)
    for i in range(n_particles):
        idx = np.argsort(particles[i])[-k_feats:]
        feats = [cols[j] for j in idx]
        try:
            pbest_scores[i] = cross_val_score(RandomForestClassifier(n_estimators=50, random_state=RANDOM_STATE), X[feats], y, cv=3).mean()
        except Exception:
            pbest_scores[i] = 0.0
    gbest_idx = int(np.argmax(pbest_scores))
    gbest = pbest[gbest_idx].copy()
    gbest_score = pbest_scores[gbest_idx]
    for t in range(iters):
        for i in range(n_particles):
            velocities[i] = 0.5*velocities[i] + 0.8*(pbest[i]-particles[i]) + 0.2*(gbest-particles[i])
            particles[i] += velocities[i]
            particles[i] = np.clip(particles[i], 0, 1)
            idx = np.argsort(particles[i])[-k_feats:]
            feats = [cols[j] for j in idx]
            try:
                score = cross_val_score(RandomForestClassifier(n_estimators=50, random_state=RANDOM_STATE), X[feats], y, cv=3).mean()
            except Exception:
                score = 0.0
            if score > pbest_scores[i]:
                pbest_scores[i] = score
                pbest[i] = particles[i].copy()
            if score > gbest_score:
                gbest_score = score
                gbest = particles[i].copy()
        print(f'PSO iter {t+1}/{iters} gbest {gbest_score:.4f}')
    idx = np.argsort(gbest)[-k_feats:]
    return [cols[j] for j in idx]


In [23]:
# Run feature selection (if target is available)
if y is None or y.isna().all():
    print('Target not available; skipping feature selection and supervised modeling.')
else:
    # Align X_enc and y (drop rows with missing target)
    mask = ~y.isna()
    Xsel = X_enc.loc[mask].reset_index(drop=True)
    ysel = y.loc[mask].reset_index(drop=True)
    print('Running RFE...')
    rfe_feats = rfe_select(Xsel, ysel, k=15)
    print('RFE selected:', rfe_feats)
    print('Running GA... (this may take several minutes)')
    ga_feats = ga_feature_search(Xsel, ysel, population_size=20, generations=6, k_best=15)
    print('GA selected:', ga_feats)
    print('Running PSO... (this may take several minutes)')
    pso_feats = pso_feature_search(Xsel, ysel, n_particles=20, iters=6, k_feats=15)
    print('PSO selected:', pso_feats)
    # Save selections
    pd.DataFrame({'RFE':rfe_feats}).to_csv(os.path.join(OUTPUT_DIR,'rfe_features.csv'), index=False)
    pd.DataFrame({'GA':ga_feats}).to_csv(os.path.join(OUTPUT_DIR,'ga_features.csv'), index=False)
    pd.DataFrame({'PSO':pso_feats}).to_csv(os.path.join(OUTPUT_DIR,'pso_features.csv'), index=False)
    print('Feature selection outputs saved to', OUTPUT_DIR)


Running RFE...
RFE selected: ['code_presentation', 'gender', 'num_of_prev_attempts', 'studied_credits', 'disability', 'clicks_total', 'days_active', 'avg_score', 'n_assessments', 'region_Scotland', 'region_Wales', 'imd_band_60-70%', 'imd_band_80-90%', 'imd_band_90-100%', 'imd_band_?']
Running GA... (this may take several minutes)
GA gen 1/6 best 0.0000
GA gen 2/6 best 0.0000
GA gen 3/6 best 0.0000
GA gen 4/6 best 0.0000
GA gen 5/6 best 0.0000
GA gen 6/6 best 0.0000
GA selected: ['num_of_prev_attempts', 'studied_credits', 'clicks_total', 'region_East Midlands Region', 'region_North Western Region', 'region_South West Region', 'region_Wales', 'imd_band_30-40%', 'imd_band_40-50%', 'imd_band_50-60%', 'imd_band_70-80%', 'imd_band_80-90%', 'imd_band_?']
Running PSO... (this may take several minutes)
PSO iter 1/6 gbest 0.0000
PSO iter 2/6 gbest 0.0000
PSO iter 3/6 gbest 0.0000
PSO iter 4/6 gbest 0.0000
PSO iter 5/6 gbest 0.0000
PSO iter 6/6 gbest 0.0000
PSO selected: ['imd_band_40-50%', 'regi

In [24]:
# Clustering: run KMeans on one-hot or numeric features to produce clusters
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score

if 'Xsel' in globals():
    Xcluster = Xsel.copy()
    # Use PCA-reduced numeric subset if too large
    if Xcluster.shape[1] > 60:
        pca = PCA(n_components=20, random_state=RANDOM_STATE)
        Xcluster_reduced = pca.fit_transform(Xcluster.select_dtypes(include=[np.number]).fillna(0))
    else:
        Xcluster_reduced = Xcluster.fillna(0).values
    db_scores = {}
    for k in range(3,11):
        km = KMeans(n_clusters=k, random_state=RANDOM_STATE, n_init=10)
        labels = km.fit_predict(Xcluster_reduced)
        db = davies_bouldin_score(Xcluster_reduced, labels)
        db_scores[k] = db
    print('Davies-Bouldin scores:', db_scores)
    best_k = min(db_scores, key=db_scores.get)
    print('Best k by DB:', best_k)
    km_final = KMeans(n_clusters=best_k, random_state=RANDOM_STATE, n_init=10)
    clusters = km_final.fit_predict(Xcluster_reduced)
    # Attach cluster labels back to original df
    df_clusters = df.loc[mask].copy()
    df_clusters['cluster'] = clusters
    df_clusters.to_csv(os.path.join(OUTPUT_DIR,'df_with_clusters.csv'), index=False)
    print('Clustered dataset saved.')
else:
    print('Xsel not available; skipping clustering.')


Davies-Bouldin scores: {3: np.float64(1.7882575732874748), 4: np.float64(1.9216819238038392), 5: np.float64(1.794201480141144), 6: np.float64(1.8340580367591812), 7: np.float64(1.9491439535474095), 8: np.float64(1.989794756362818), 9: np.float64(1.9922310828939658), 10: np.float64(1.9281758364403605)}
Best k by DB: 3
Clustered dataset saved.


In [25]:
# Supervised models: RandomForest, KNN, XGBoost (if available)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

if 'Xsel' in globals():
    Xtrain, Xtest, ytrain, ytest = train_test_split(Xsel, ysel, test_size=0.2, random_state=RANDOM_STATE, stratify=ysel)
    models_results = []
    # Random Forest
    rf = RandomForestClassifier(n_estimators=200, random_state=RANDOM_STATE)
    rf.fit(Xtrain, ytrain)
    preds = rf.predict(Xtest)
    acc = accuracy_score(ytest, preds)
    print('RF test acc:', acc)
    models_results.append({'model':'RandomForest','accuracy':acc})
    # KNN
    from sklearn.neighbors import KNeighborsClassifier
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(Xtrain, ytrain)
    acc_knn = accuracy_score(ytest, knn.predict(Xtest))
    print('KNN acc:', acc_knn)
    models_results.append({'model':'KNN','accuracy':acc_knn})
    # XGBoost if available
    try:
        import xgboost as xgb
        xclf = xgb.XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=RANDOM_STATE)
        xclf.fit(Xtrain, ytrain)
        acc_xgb = accuracy_score(ytest, xclf.predict(Xtest))
        print('XGBoost acc:', acc_xgb)
        models_results.append({'model':'XGBoost','accuracy':acc_xgb})
    except Exception as e:
        print('XGBoost not available or failed:', e)
    pd.DataFrame(models_results).to_csv(os.path.join(OUTPUT_DIR,'model_benchmarks.csv'), index=False)
    print('Model benchmarks saved to', OUTPUT_DIR)
else:
    print('No Xsel; cannot train supervised models.')


RF test acc: 0.6916705016106764
KNN acc: 0.613591041570793
XGBoost acc: 0.7024083448381654
Model benchmarks saved to /content/outputs


In [26]:
# Simple feed-forward neural networks with multiple architectures
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

if 'Xsel' in globals():
    # For NN, convert y to 0..n-1 (already numeric) and use sparse_categorical_crossentropy
    input_dim = Xtrain.shape[1]
    num_classes = len(np.unique(ytrain))
    def build_ffnn(hidden_layers=2, nodes=32, lr=0.001):
        model = Sequential()
        model.add(Dense(nodes, activation='relu', input_shape=(input_dim,)))
        for _ in range(hidden_layers-1):
            model.add(Dense(nodes, activation='relu'))
            model.add(Dropout(0.2))
        model.add(Dense(num_classes, activation='softmax'))
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        return model
    nn_configs = [(1,32),(2,32),(3,16)]
    nn_results = []
    for hl, nodes in nn_configs:
        model = build_ffnn(hidden_layers=hl, nodes=nodes)
        history = model.fit(Xtrain, ytrain, validation_split=0.15, epochs=30, batch_size=512, verbose=0)
        loss, acc = model.evaluate(Xtest, ytest, verbose=0)
        print(f'NN hl={hl} nodes={nodes} test acc={acc:.4f}')
        nn_results.append({'config':f'hl{hl}_n{nodes}','accuracy':acc})
        # Save training loss plot
        fig = plt.figure(figsize=(6,3))
        plt.plot(history.history['loss'], label='loss')
        plt.plot(history.history['val_loss'], label='val_loss')
        plt.legend()
        plt.title('Training loss - ' + f'hl{hl}_n{nodes}')
        fig.savefig(os.path.join(OUTPUT_DIR, f'nn_loss_hl{hl}_n{nodes}.png'), bbox_inches='tight')
        plt.close(fig)
    pd.DataFrame(nn_results).to_csv(os.path.join(OUTPUT_DIR,'nn_results.csv'), index=False)
    print('NN results saved.')
else:
    print('Xsel not available; skipping neural networks.')


NN hl=1 nodes=32 test acc=0.6630
NN hl=2 nodes=32 test acc=0.6710
NN hl=3 nodes=16 test acc=0.6549
NN results saved.


In [ ]:
# XAI: SHAP (TreeExplainer) and optional LIME
try:
    import shap
    print('SHAP available. Computing SHAP values for RandomForest...')
    explainer = shap.TreeExplainer(rf)
    # For multiclass, shap_values is a list; we will plot summary for each class
    shap_values = explainer.shap_values(Xtest)
    # Summary plot for class 2 (Pass) if exists
    try:
        cls = 2 if len(shap_values)>2 else 0
        shap.summary_plot(shap_values[cls], Xtest, plot_type='bar', show=False)
        plt.gcf().savefig(os.path.join(OUTPUT_DIR,'shap_summary_class.png'), bbox_inches='tight')
        plt.close()
        print('SHAP summary saved.')
    except Exception as e:
        print('SHAP plotting failed:', e)
except Exception as e:
    print('SHAP not installed or failed:', e)

# LIME (optional)
try:
    from lime.lime_tabular import LimeTabularExplainer
    print('LIME available. Producing a local explanation...')
    expl = LimeTabularExplainer(Xtrain.values, feature_names=Xtrain.columns.tolist(), class_names=[str(c) for c in np.unique(ytrain)], mode='classification')
    i = 10 if Xtest.shape[0]>10 else 0
    exp = expl.explain_instance(Xtest.iloc[i].values, rf.predict_proba, num_features=10)
    # Save explanation as HTML
    html = exp.as_html()
    open(os.path.join(OUTPUT_DIR,'lime_explanation.html'),'w').write(html)
    print('LIME explanation saved to', os.path.join(OUTPUT_DIR,'lime_explanation.html'))
except Exception as e:
    print('LIME not installed or failed:', e)


SHAP available. Computing SHAP values for RandomForest...


## Notebook Complete

All results (figures, CSVs, explanations) are saved to the `outputs` directory.